In [ ]:
#data formatting
QUORA_PATH="/home/dc/cs230_project/dataset"

def clean_quora(quora_path):
    '''
    input: path of quora tsv file downloaded from kaggle
    output: df with questions <10 chars removed
    
    '''
    df = pd.read_csv(os.path.join(quora_path,"quora_duplicate_questions.tsv"),sep="\t")
    print(df.head())
    df = df.drop(["id","qid1","qid2"],axis=1)
    print(df.count())
    df=df.dropna()
    print(df.count())
    df['q1_len'] = df['question1'].apply(len)
    df['q2_len'] = df['question2'].apply(len)
    print(df.head())
    #print(df.loc[df['q1_len'] < 10])
    #print(df.loc[df['q2_len'] < 10])
    df = df.loc[ (df['q1_len'] > 10) & (df['q2_len'] > 10)]
    print(df.count())
    return df


def load_single_file(filename):
    fh = open(os.path.join(params.data_dir,filename+'.pkl'),'rb')
    data = pickle.load(fh)
    fh.close()
    return data

"""
def load_data():
    '''
    remove <10 char length question rows. this isnt enough cleaning.
    '''
    X_train = load_single_file("X_train")
    X_valid = load_single_file("X_valid")
    X_test = load_single_file("X_test")
    y_train = load_single_file("y_train")
    y_valid = load_single_file("y_valid")
    y_test = load_single_file("y_test")
    return X_train, X_valid, X_test, y_train,y_valid, y_test
"""    
   
    
def format_data(X_train, X_valid,X_test, y_train,y_valid,y_test):
    s1 = {}
    s2 = {}
    target = {}
    s1['train'],s1['dev'],s1['test'],s2['train'],s2['dev'],s2['test'] = {},{},{},{},{},{}
    target['train'],target['dev'],target['test']={},{},{}

    s1['train']['sent'] = [x for x in X_train[:,0]]
    s2['train']['sent'] = [x for x in X_train[:,1]]
    s1['dev']['sent'] = [x for x in X_valid[:,0]]
    s2['dev']['sent'] = [x for x in X_valid[:,1]]
    s1['test']['sent'] = [x for x in X_test[:,0]]
    s2['test']['sent'] = [x for x in X_test[:,1]]
    target['train']['data'] = np.array([x[0] for x in y_train])
    target['dev']['data'] = np.array([x[0] for x in y_valid])
    target['test']['data'] = np.array([x[0] for x in y_test.tolist()])

    train = {'s1': s1['train']['sent'], 's2': s2['train']['sent'],
             'label': target['train']['data']}
    dev = {'s1': s1['dev']['sent'], 's2': s2['dev']['sent'],
           'label': target['dev']['data']}
    test = {'s1': s1['test']['sent'], 's2': s2['test']['sent'],
            'label': target['test']['data']}
    return train,dev,test



def get_word_dict(sentences):
    # create vocab of words
    word_dict = {}
    for sent in sentences:
        for word in sent.split():
            if word not in word_dict:
                word_dict[word] = ''
    word_dict['<s>'] = ''
    word_dict['</s>'] = ''
    word_dict['<p>'] = ''
    return word_dict


def get_glove(word_dict, glove_path):
    # create word_vec with glove vectors
    word_vec = {}
    with open(glove_path) as f:
        for line in f:
            word, vec = line.split(' ', 1)
            if word in word_dict:
                word_vec[word] = np.array(list(map(float, vec.split())))
    print('Found {0}(/{1}) words with glove vectors'.format(
                len(word_vec), len(word_dict)))
    return word_vec


def build_vocab(sentences, glove_path):
    word_dict = get_word_dict(sentences)
    word_vec = get_glove(word_dict, glove_path)
    print('Vocab size : {0}'.format(len(word_vec)))
    return word_vec


def quora(big=False,small=False,clean=False):
    
    
    X_train,X_valid,X_test,y_train,y_valid,y_test = load_data(big=big,small=small,clean=clean)
    
    train,valid,test = format_data(X_train, X_valid,X_test, y_train,y_valid,y_test)
    print(f"quora checkpoint len(train[s1]):{len(train['s1'])},len(train[s2]):{len(train['s2'])},\
          len(train[label]):{len(train['label'])}")
    print('============')
    print(f"len(valid['s1']):{len(valid['s1'])}, len(valid[s2]):{len(valid['s2'])}, \
          len(valid['label']):{len(valid['label'])}")
    print('============')
    print(f"len(test['s1']):{len(test['s1'])},len(test['s2']):{len(test['s2'])}, \
          len(test['label']):{len(test['label'])}")
          
    word_vec = build_vocab(train['s1'] + train['s2'] +
                       valid['s1'] + valid['s2'] +
                       test['s1'] + test['s2'], W2V_PATH)
    for split in ['s1', 's2']:
        for data_type in ['train', 'valid', 'test']:
            eval(data_type)[split] = np.array([['<s>'] +
            [word for word in sent.split() if word in word_vec] +
            ['</s>'] for sent in eval(data_type)[split]])
    
    return train,valid,test,word_vec
